# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '85.9.KQJT873.QJT'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '3D', 'insta_score': 0.979}]
[{'call': '3D', 'insta_score': 0.968}]
[{'call': '3D', 'insta_score': 0.979}]
[{'call': '3D', 'insta_score': 0.968}]


In [3]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 1.0, 'expected_score': 392, 'adjustment': 50},
 {'call': '5D',
  'insta_score': 0.0,
  'expected_score': -450,
  'adjustment': -1751}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['AQJx.AKTxx..A8xx Txx.QJ8.A9xxxx.K K9xx.xxxx..9xxxx 8x.9.KQJT8xx.QJT 0.05584 \n 2D-P-P-X-P-P-P (9.6)  \n 2D-P-P-X-P-P-5D-X-P-P-P (9.6) ',
 'AKQJT.JT8xx..AKx xx.Axx.A9xxxx.xx 9xxx.KQxx..98xxx 8x.9.KQJT8xx.QJT 0.02452 \n 2D-P-P-X-P-P-P (9.8)  \n 2D-P-P-X-P-P-5D-X-P-P-P (9.8) ',
 'AT9x.KJxx..AKxxx xx.Qxx.A9xxxx.8x KQJxx.AT8xx..9xx 8x.9.KQJT8xx.QJT 0.02050 \n 2D-P-P-X-P-P-P (8.5)  \n 2D-P-P-X-P-P-5D-X-P-P-P (8.5) ',
 'AK9xx.AQJxxx..Ax Jx.Kx.A9xxxx.9xx QTxx.T8xx..K8xxx 8x.9.KQJT8xx.QJT 0.01909 \n 2D-P-P-X-P-P-P (8.5)  \n 2D-P-P-X-P-P-5D-X-P-P-P (8.5) ',
 'AKTx.KJxx..Axxxx J9x.QTx.A9xxxx.8 Qxxx.A8xxx..K9xx 8x.9.KQJT8xx.QJT 0.01766 \n 2D-P-P-X-P-P-P (9.2)  \n 2D-P-P-X-P-P-5D-X-P-P-P (9.2) ',
 'AK9x.AKQJx..AKxx QJx.xx.A9xxxx.xx Txxx.T8xxx..98xx 8x.9.KQJT8xx.QJT 0.01453 \n 2D-P-P-X-P-P-P (8.7)  \n 2D-P-P-X-P-P-5D-X-P-P-P (8.7) ',
 'AJxxxx.AQJx..AKx T.KTx.A9xxxx.8xx KQ9x.8xxxx..9xxx 8x.9.KQJT8xx.QJT 0.01324 \n 2D-P-P-X-P-P-P (9.5)  \n 2D-P-P-X-P-P-5D-X-P-P-P (9.5) ',
 'AK9xxx.KQx..AKxx JT.Ax.A9

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

8x.9.KQJT8xx.QJT x.J8xxx.9xxx.Kxx AKQJ9.AKxx..A9xx Txxxx.QTx.Ax.8xx
8x.9.KQJT8xx.QJT K9xx.J8xx.9x.8xx AQJTx.AKTx..AKxx xx.Qxxx.Axxx.9xx
8x.9.KQJT8xx.QJT Tx.QJxx.9x.98xxx AKQ9xx.K8xx.Ax.A Jxx.ATxx.xx.Kxxx
8x.9.KQJT8xx.QJT KQ9.xxx.xx.xxxxx AJxxxx.AKT8.A9.K Tx.QJxxx.xx.A98x
8x.9.KQJT8xx.QJT 9.xxxx.9xxx.K8xx AKQJTx.AQ8x.Ax.x xxxx.KJTx..A9xxx
8x.9.KQJT8xx.QJT JTx.8xx.9x.Axxxx AKQxx.AKQJ.Axx.x 9xx.Txxxx.x.K98x
8x.9.KQJT8xx.QJT J9x.QJ8xx.x.9xxx AKQTxx.AKTx.9x.A xx.xxx.Axx.K8xxx
8x.9.KQJT8xx.QJT J9x.T8xx.A9xx.8x AKxxx.AKJx..AK9x QTx.Qxxx.xx.xxxx
8x.9.KQJT8xx.QJT x.JT8x.9x.Kxxxxx KQJxx.AKxx.Axx.A AT9xx.Qxxx.x.98x
8x.9.KQJT8xx.QJT Txx.K8xxx.9xx.8x AKQJ9x.AQJx.A.xx xx.Txx.xx.AK9xxx
